In [7]:
import numpy as np
import pandas as pd

In [8]:
df = pd.read_csv('C:/Users/sapta/Downloads/Sales_Dataset.csv')


In [9]:
df

,Customer ID,Customer Name,Date,Product Name,Category,Quantity,Unit Price,Total Price,City
0,7166,Suman,2025-03-05,Flavored Condom,Condoms,2,200,400,Delhi
1,3562,Vikram,2025-02-17,Cock Ring,Sex Toys,2,800,1600,Ahmedabad
2,9249,Neha,2025-03-01,Anal Beads,Sex Toys,3,1500,4500,Mumbai
3,9249,Neha,2025-03-01,Sports Bra,Female Undergarments,2,700,1400,Mumbai
4,9249,Neha,2025-03-01,Ultra Thin Condom,Condoms,3,250,750,Mumbai
...,...,...,...,...,...,...,...,...,...
237,3772,Vikram,2025-02-15,Anal Beads,Sex Toys,2,1500,3000,Bangalore
238,9780,Priya,2025-03-12,Mini Skirt,Bold Sexy Female Dress,1,700,700,Hyderabad
239,9780,Priya,2025-03-12,Lace Bra,Female Undergarments,1,500,500,Hyderabad
240,3624,Priya,2025-02-27,Padded Bra,Female Undergarments,3,800,2400,Mumbai


In [ ]:
# Load the CSV as a structured array.
# Adjust the delimiter, encoding, and dtype if necessary.
data = np.genfromtxt('C:/Users/sapta/Downloads/Sales_Dataset.csv', delimiter=',', names=True, dtype=None, encoding='utf-8')

# Convert the "Date" column to datetime64[D]
dates = data['Date'].astype('datetime64[D]')

# Earliest and latest dates
earliest_date = dates.min()
latest_date = dates.max()

# Unique purchase days
unique_dates = np.unique(dates)
num_unique_days = unique_dates.size

print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)
print("Number of Unique Purchase Days:", num_unique_days)


Earliest Date: 2025-02-11
Latest Date: 2025-03-12
Number of Unique Purchase Days: 28


i: Monthly Sales Analysis

In [10]:
# Extract year-month from dates (using string slicing on ISO formatted dates)
year_month = dates.astype('datetime64[M]')  # This gives the month resolution

# Compute total quantity sold per month
unique_months, month_indices = np.unique(year_month, return_inverse=True)
# Sum quantities for each unique month
quantities = data['Quantity'].astype(np.float64)
total_quantity_per_month = np.zeros(unique_months.size)
for i in range(unique_months.size):
    total_quantity_per_month[i] = quantities[month_indices == i].sum()

# Total revenue per month (for Total Price)
total_price = data['Total_Price'].astype(np.float64)
total_revenue_per_month = np.zeros(unique_months.size)
for i in range(unique_months.size):
    total_revenue_per_month[i] = total_price[month_indices == i].sum()

# Which month had the highest revenue?
max_rev_index = np.argmax(total_revenue_per_month)
best_month = unique_months[max_rev_index]

print("Unique Months:", unique_months)
print("Total Quantity per Month:", total_quantity_per_month)
print("Total Revenue per Month:", total_revenue_per_month)
print("Month with Highest Revenue:", best_month)


Unique Months: ['2025-02' '2025-03']
Total Quantity per Month: [268. 229.]
Total Revenue per Month: [263600. 246950.]
Month with Highest Revenue: 2025-02


ii: Product Category Performance

In [11]:
# Get unique categories
categories, category_indices = np.unique(data['Category'], return_inverse=True)

# Initialize an array to hold the average sale per transaction per category
avg_sale_per_category = np.zeros(categories.size)

for i in range(categories.size):
    cat_mask = (category_indices == i)
    # Compute average for the category
    avg_sale_per_category[i] = total_price[cat_mask].mean()

# Identify the category with the highest average sale
max_avg_index = np.argmax(avg_sale_per_category)
best_category = categories[max_avg_index]

print("Categories:", categories)
print("Average Sale per Category:", avg_sale_per_category)
print("Category with Highest Average Sale:", best_category)


Categories: ['Bold Sexy Female Dress' 'Condoms' 'Female Undergarments' 'Sex Toys']
Average Sale per Category: [2929.8245614   612.5        1453.22580645 3439.68253968]
Category with Highest Average Sale: Sex Toys


iii: Top 3 Cities by Revenue

In [13]:
# Get unique cities
cities, city_indices = np.unique(data['City'], return_inverse=True)

total_revenue_per_city = np.zeros(cities.size)
for i in range(cities.size):
    total_revenue_per_city[i] = total_price[city_indices == i].sum()

# Get indices for top 3 cities
top3_indices = np.argsort(total_revenue_per_city)[-3:][::-1]
top3_cities = cities[top3_indices]
top3_revenues = total_revenue_per_city[top3_indices]

print("Total Revenue per City:", dict(zip(cities, total_revenue_per_city)))
print("Top 3 Cities by Revenue:", list(zip(top3_cities, top3_revenues)))


Total Revenue per City: {np.str_('Ahmedabad'): np.float64(44100.0), np.str_('Bangalore'): np.float64(60500.0), np.str_('Chennai'): np.float64(39450.0), np.str_('Delhi'): np.float64(47100.0), np.str_('Hyderabad'): np.float64(33300.0), np.str_('Jaipur'): np.float64(44800.0), np.str_('Kolkata'): np.float64(108400.0), np.str_('Lucknow'): np.float64(58600.0), np.str_('Mumbai'): np.float64(44050.0), np.str_('Pune'): np.float64(30250.0)}
Top 3 Cities by Revenue: [(np.str_('Kolkata'), np.float64(108400.0)), (np.str_('Bangalore'), np.float64(60500.0)), (np.str_('Lucknow'), np.float64(58600.0))]


iv: Customer Lifetime Value (CLV) Calculation

In [16]:
# Get unique customer IDs
customers, customer_indices = np.unique(data['Customer_ID'], return_inverse=True)

num_customers = customers.size
total_revenue_per_customer = np.zeros(num_customers)
transaction_count_per_customer = np.zeros(num_customers)

for i in range(num_customers):
    mask = (customer_indices == i)
    total_revenue_per_customer[i] = total_price[mask].sum()
    transaction_count_per_customer[i] = mask.sum()

# Identify top 5 customers by revenue
top5_indices = np.argsort(total_revenue_per_customer)[-5:][::-1]
top5_customers = customers[top5_indices]
top5_revenues = total_revenue_per_customer[top5_indices]

print("Top 5 Customers by Lifetime Revenue:")
for cid, rev in zip(top5_customers, top5_revenues):
    print("Shape of data:", data.shape)  # structured array overall shape
print("Length of Customer_ID column:", data["Customer_ID"].shape)
print("Length of Total_Price column:", data["Total_Price"].shape)

# If these do not match, you know the problem is in how the CSV is parsed.


Top 5 Customers by Lifetime Revenue:
Shape of data: (242,)
Shape of data: (242,)
Shape of data: (242,)
Shape of data: (242,)
Shape of data: (242,)
Length of Customer_ID column: (242,)
Length of Total_Price column: (242,)


v: High-Value Transactions Detection

In [17]:
# Calculate the 90th percentile for Total Price
percentile_90 = np.percentile(total_price, 90)

# Create a mask for high-value transactions
high_value_mask = total_price > percentile_90
num_high_value = high_value_mask.sum()

# Revenue from high-value transactions and percentage of total revenue
high_value_revenue = total_price[high_value_mask].sum()
percentage_revenue = (high_value_revenue / total_price.sum()) * 100

print("90th Percentile Value:", percentile_90)
print("Number of High-Value Transactions:", num_high_value)
print("High-Value Revenue Percentage: {:.2f}%".format(percentage_revenue))


90th Percentile Value: 5900.0000000000055
Number of High-Value Transactions: 25
High-Value Revenue Percentage: 35.49%
